In [ ]:
# 라이브러리 import
import pandas as pd
import numpy as np

from selenium import webdriver
from bs4 import BeautifulSoup 
from tqdm.notebook import tqdm
import time
import re

# url 수집 코드 

In [ ]:
driver = webdriver.Chrome('C:\chrome\chromedriver.exe')
url = "https://www.idus.com/oc" # 온라인
driver.get(url)
time.sleep(3)

time.sleep(2)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 카테고리 클릭
driver.find_element_by_xpath("/html/body/div[3]/div[3]/section[6]/div/div/div[3]/div/div[2]/button/span").click( )
time.sleep(1)

# 공예부분 모두 클릭
for i in range(13):
  driver.find_element_by_xpath("/html/body/div[3]/div[3]/section[6]/div/div/div[3]/div[2]/div[1]/div[2]/div[1]/div[2]/div[{}]/input".format(i+1)).click( )

# 스크롤을 밑으로 내려주는 함수
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0, 19431049)")
    time.sleep(1)

# n: 스크롤할 횟수 설정
n = 5
i = 0
while i < n: # 이 조건이 만족되는 동안 반복 실행
    scroll_down(driver) # 스크롤 다운
    i = i+1
    
url_data = []

# 크롤링한 url 정제 시작. 문자열에 대해서는 정규표현식 사용하여 전처리 진행
for target in target_raw:
    url = target.find_elements_by_css_selector('div > a')[0].get_attribute('href')
    category = target.find_elements_by_css_selector('a > div.card-info.class-info > div.label')[0].text
    category = re.sub(r"[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z|0-9|.|,| ]", " ", title)
    title = target.find_elements_by_css_selector('a > div.card-info.class-info > div.class-name')[0].text
    title = re.sub(r"[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z|0-9|.|,| ]", " ", title)
    url_data.append([url, category, title])
url_data
columns= ['url', 'category', 'title']
df = pd.DataFrame(url_data, columns = columns)  # 수집한 데이터들을 바탕으로 데이터프레임 구축

# 데이터 프레임 저장하기
df.to_excel('./idus/idus_url_online.xlsx', index=False)
df

In [ ]:
url_df=pd.read_excel('./idus/idus_url_online.xlsx', index=False)
url_df

,url,category,title
0,https://www.idus.com/oc/class/230,"도자기, 내 공간을 완성하는 갖고 싶은 접시 만들기","명품 브랜드에서 줄줄이 내놓는 핫아이템, 라탄 트래블백 클래스"
1,https://www.idus.com/oc/class/131,"명품 브랜드에서 줄줄이 내놓는 핫아이템, 라탄 트래블백 클래스",따뜻한 감성으로 가득 채우는 라탄 소품
2,https://www.idus.com/oc/class/20,따뜻한 감성으로 가득 채우는 라탄 소품,"반짝반짝 투명한 매력, 누구나 쉽게 배우는 레진공예"
3,https://www.idus.com/oc/class/145,"반짝반짝 투명한 매력, 누구나 쉽게 배우는 레진공예","해외에서도 핫한 K 보자기, 종이 보자기 선물포장 수업"
4,https://www.idus.com/oc/class/187,"해외에서도 핫한 K 보자기, 종이 보자기 선물포장 수업","내 손 안의 바다, 바다를 닮은 레진 소품"
...,...,...,...
69,https://www.idus.com/oc/class/116,"여행의 즐거운 기억과 함께 하는, 페이퍼아트 집 만들기","펠트에 담아낸 보들보들 귀여운 일러스트, 바느질 클래스"
70,https://www.idus.com/oc/class/115,"펠트에 담아낸 보들보들 귀여운 일러스트, 바느질 클래스","인생신발, 에스파듀 슬립온 가죽공예 수제화"
71,https://www.idus.com/oc/class/77,"인생신발, 에스파듀 슬립온 가죽공예 수제화","기다림 마저 설레게 하는 하드 왁스 카빙 주얼리, 금속공예"
72,https://www.idus.com/oc/class/64,"기다림 마저 설레게 하는 하드 왁스 카빙 주얼리, 금속공예",나만의 색감과 각인으로 채워 넣는 내 방 안 가죽 공방


# 리뷰, 댓글 수집 코드

In [ ]:
import time
import re
import pandas as pd
from tqdm.notebook import tqdm 

# url 불러오기
url_load = pd.read_excel('./idus/idus_url_online.xlsx',engine='openpyxl')  # 수집한 url 불러오기
num_list = len(url_load)
print(num_list) # 총 url 갯수
url_load

start = time.time() 

title_list = []
nickname_list = []
datetime_list = []
like_list = []
content_list = []
comment_nickname = []
comment_content = []

# 수집한 url 돌면서 데이터 수집
for i in tqdm(range(0, num_list)):
    driver = webdriver.Chrome('C:\chrome\chromedriver.exe')  # ChromeDriver 경로
    driver.get(url_load.url[i])
    time.sleep(2)
    
    try:
        
        target_info = {}
        target_info_co = {} 
        
        # 수강 후기 부분 -------------------------------------
        try:
             # 수강 후기 클릭
            driver.find_element_by_link_text("수강 후기").click( )
            time.sleep(2)

            # n: 반복할 횟수 설정, '더보기' 클릭
            n = 6
            i = 0
            while i < n: # 이 조건이 만족되는 동안 반복 실행
                try:
                    time.sleep(2)
                    driver.find_element_by_css_selector("#edu-review > div.pagination > div > button > span").click( ) #더보기
                    i = i+1
                except:
                    break

        except:
            pass

        # 수집한 데이터 중 문자열 데이터에 대해 정규표현식 사용하여 전처리 진행
        reviews = driver.find_elements_by_css_selector('ul.list-style-review>li')
        for review in reviews:                     
            title = driver.find_element_by_css_selector("h2.sticky_aside__title").text        
            title = re.sub(r"[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z|0-9| ]", "", title)
            title_list.append(title)

            nickname = review.find_elements_by_css_selector('a > div.clf > div > div.txt-block > em')[0].text
            nickname = re.sub(r"[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z|0-9|.|,| ]", "", nickname)
            nickname_list.append(nickname)

            datetime = review.find_elements_by_css_selector('a > div.clf > div > div.txt-block > time')[0].text
            datetime_list.append(datetime)

            like = review.find_elements_by_css_selector('a > div.clf > span')[0].get_attribute('data-value')
            like_list.append(like)

            content = review.find_elements_by_css_selector('a > div.area-txt > div > span')[0].text
            content = re.sub(r"[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z|0-9|.|,| ]", " ", content) 
            content_list.append(content)
        
        
        # 댓글 부분 ------------------------------------------
        try:
             # 댓글 클릭
            driver.find_element_by_link_text("댓글").click( )
            time.sleep(2)

            # n: 반복할 횟수 설정, '더보기' 클릭
            n = 15
            i = 0
            while i < n: # 이 조건이 만족되는 동안 반복 실행
                try:
                    time.sleep(2)
                    driver.find_element_by_css_selector("#edu-comment > div.comment_form > div > button > span").click( ) #이전 댓글 더보기
                    i = i+1
                except:
                    break

        except:
            pass 
        
        comments = driver.find_elements_by_css_selector('ul.comment-ui>li') 
        for comment in comments:                     
            comment_nick =comment.find_elements_by_css_selector('div.comment-block > div > span')[0].text
            comment_nick = re.sub(r"[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z|0-9|.|,| ]", "", comment_nick)
            comment_nickname.append(comment_nick)

            comment_con = comment.find_elements_by_css_selector(' div.comment-block > div > p')[0].text
            comment_con = re.sub(r"[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣|A-Z|a-z|0-9|.|,| ]", " ", comment_con)
            comment_content.append(comment_con)           

        # 크롤링이 성공하면 글 제목을 출력하고 브라우저 종료
        print(title)
        driver.close()
    
    # 에러나면 현재 크롬창을 닫고 다음 글(i+1)로 이동
    except Exception as e:
        driver.close()
        print('실패', e, type(e))
        continue
        
end = time.time() - start
print("used time: %dm%ds" % divmod(end,60))
        
target_info['title'] = title_list
target_info['nickname'] = nickname_list
target_info['datetime'] = datetime_list
target_info['like'] = like_list
target_info['content'] = content_list
target_info_co['comment_nickname'] = comment_nickname
target_info_co['comment_content'] = comment_content



74


  0%|          | 0/74 [00:00<?, ?it/s]

명품 브랜드에서 줄줄이 내놓는 핫아이템 라탄 트래블백 클래스
따뜻한 감성으로 가득 채우는 라탄 소품
반짝반짝 투명한 매력 누구나 쉽게 배우는 레진공예
해외에서도 핫한 K보자기 종이보자기 선물포장 수업
내 손 안의 바다 바다를 닮은 레진 소품
특별한 날 하나뿐인 선물 메시지 플라워 꽃풍선 만들기
니들펠트로 만드는 개성 가득 괴물 친구들
내 손으로 직접 피워내는 꽃 페이퍼 플라워 클래스
내 손으로 직접 피워내는 꽃 페이퍼 플라워 클래스
코로나 블루 극복 생기 한방울 더해줄 플라워 클래스
복을 선물한다는 의미가 담긴 보자기 포장
일상을 달콤하게 채울 영롱한 디저트 캔들
천년의 빛을 조각하는 자개공예 클래스
닥나무로부터 피어난 천년의 꽃 한지꽃 클래스
영롱한 하루를 만들어 드립니다 빛나는 레진 공예
깎을수록 뚜렷해지는 나의 취향 우드 목조각 목공 클래스
한땀 한땀 섬세하게 만들어가는 가죽소품 클래스
나전칠기의 재해석 일상 속 특별한 자개 소품 만들기
디자인을 입힌 나만의 캔들 만들기
감성저격 물성형 입체 가죽공예
드르륵 재봉틀로 일상 속 패브릭 소품 만들기
방 안을 가득 채우는 여름 라탄 클래스
바느질로 만드는 화사한 헤어 액세서리
종이가 건네주는 위로 페이퍼 커팅 클래스
나무로 만드는 아침 살림 우드카빙 브런치 세트
사랑스런 아이들의 그림으로 만드는 입체인형 재봉 클래스
작은 미니어처들을 모아 집을 짓는 돌하우스 공예
내 맘대로 그려보는 마카쥬 가죽지갑 가죽공예
펀치니들 클래스로 만나는 보태니컬 홈 인테리어
우연한 설렘 레진 플루이드 아트 서빙보드와 코스터
오직 하나뿐인 패턴 만들기 폴리머클레이
누구나 하나 쯤은 만들어 보고 싶은 가죽가방 사첼백 클래스
자연스레 섞여가는 색의 어울림 레진 마블링 클래스
톱질만 잘 해도 금속공예 마스터 클래스
향기로 풍경을 그려내는 뷰티 조향 기초 향수 클래스
감각적인 텍스처와 천연 원석의 실버 주얼리 금속 공예 클래스
빛을 담아 간직하는 유리공예 스테인드 글라스
빛을 담아 간직하는 유리공예 스테인드 글라스
손쉽게 꼼지락 꼼지락 재봉

In [ ]:
import pandas as pd
columns = ['title', 'nickname', 'datetime', 'like', 'content'] # review 부분
result_df = pd.DataFrame(target_info,columns=columns)
result_df

,title,nickname,datetime,like,content
0,명품 브랜드에서 줄줄이 내놓는 핫아이템 라탄 트래블백 클래스,민영,2021년 5월 20일,5,
1,명품 브랜드에서 줄줄이 내놓는 핫아이템 라탄 트래블백 클래스,수디닝,2021년 5월 19일,5,
2,명품 브랜드에서 줄줄이 내놓는 핫아이템 라탄 트래블백 클래스,송다현,2021년 5월 19일,5,
3,명품 브랜드에서 줄줄이 내놓는 핫아이템 라탄 트래블백 클래스,강미선,2021년 5월 17일,5,
4,명품 브랜드에서 줄줄이 내놓는 핫아이템 라탄 트래블백 클래스,오윤서,2021년 5월 17일,5,ㅎ
...,...,...,...,...,...
1664,도자기 내 공간을 완성하는 갖고 싶은 접시 만들기,최아진,2021년 1월 20일,5,
1665,도자기 내 공간을 완성하는 갖고 싶은 접시 만들기,okjo337,2020년 12월 27일,5,
1666,도자기 내 공간을 완성하는 갖고 싶은 접시 만들기,박민희,2020년 12월 21일,4,
1667,도자기 내 공간을 완성하는 갖고 싶은 접시 만들기,정은주,2020년 12월 20일,5,


# 엑셀 저장

In [ ]:
import pandas as pd
columns = ['title', 'nickname', 'datetime', 'like', 'content'] # review 부분
result_df = pd.DataFrame(target_info,columns=columns)
result_df
result_df.to_excel('./idus/review_result_online.xlsx',encoding='utf-8-sig', index=False)